In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!unzip -q "/content/drive/MyDrive/brain_tumor_dataset.zip" -d "/content/brain_tumor_dataset"


In [3]:
import os, shutil
import pandas as pd
from sklearn.model_selection import train_test_split

raw_path = "/content/brain_tumor_dataset"
output_path = "/content/brain_mri_split"
classes = ['yes', 'no']

data = []
for cls in classes:
    cls_dir = os.path.join(raw_path, cls)
    for fname in os.listdir(cls_dir):
        if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            data.append({"path": os.path.join(cls_dir, fname), "label": cls})

df = pd.DataFrame(data)

train_df, temp_df = train_test_split(df, test_size=0.3, stratify=df["label"], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df["label"], random_state=42)

splits = {"train": train_df, "val": val_df, "test": test_df}

for split, split_df in splits.items():
    for cls in classes:
        split_cls_dir = os.path.join(output_path, split, cls)
        os.makedirs(split_cls_dir, exist_ok=True)
        for _, row in split_df[split_df["label"] == cls].iterrows():
            shutil.copy(row["path"], os.path.join(split_cls_dir, os.path.basename(row["path"])))


In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

data_dir = "/content/brain_mri_split"

train_ds = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform)
val_ds = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transform)
test_ds = datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=transform)

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=0)
val_loader = DataLoader(val_ds, batch_size=8, shuffle=False, num_workers=0)
test_loader = DataLoader(test_ds, batch_size=8, shuffle=False, num_workers=0)


In [5]:
import torch
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class_labels = np.unique(train_ds.targets)
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=class_labels,
                                     y=np.array(train_ds.targets))

weights = torch.tensor(class_weights, dtype=torch.float).to(device)


In [6]:
import timm
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

model_swin = timm.create_model(
    'swin_base_patch4_window7_224',
    pretrained=True,
    num_classes=2
)
model_swin.to(device)

criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = optim.Adam(model_swin.parameters(), lr=3e-4)

for epoch in range(10):
    model_swin.train()
    total_loss = 0
    for imgs, labels in tqdm(train_loader, desc=f"Swin Epoch {epoch+1}"):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model_swin(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss:.4f}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

Swin Epoch 1: 100%|██████████| 23/23 [06:40<00:00, 17.39s/it]


Epoch 1 Loss: 19.1955


Swin Epoch 2: 100%|██████████| 23/23 [06:47<00:00, 17.71s/it]


...

Epoch 8 Loss: 15.9398


Swin Epoch 9: 100%|██████████| 23/23 [06:45<00:00, 17.63s/it]


Epoch 9 Loss: 16.3038


Swin Epoch 10: 100%|██████████| 23/23 [06:47<00:00, 17.74s/it]

Epoch 10 Loss: 16.0188


In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import torch

model_swin.eval()

y_true = []
y_pred = []

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model_swin(imgs)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())


In [8]:
# Accuracy
acc = accuracy_score(y_true, y_pred)

# Precision
prec = precision_score(y_true, y_pred, zero_division=0)

# Recall
rec = recall_score(y_true, y_pred, zero_division=0)

# F1-score
f1 = f1_score(y_true, y_pred, zero_division=0)

# Classification Report
report = classification_report(y_true, y_pred)

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)

print("📋 Classification Report:\n", report)
print("🎯 Accuracy:", acc)
print("🎯 Precision:", prec)
print("🎯 Recall:", rec)
print("🎯 F1 Score:", f1)
print("🌀 Confusion Matrix:\n", cm)


📋 Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        15
           1       0.61      1.00      0.75        23

    accuracy                           0.61        38
   macro avg       0.30      0.50      0.38        38
weighted avg       0.37      0.61      0.46        38

🎯 Accuracy: 0.6052631578947368
🎯 Precision: 0.6052631578947368
🎯 Recall: 1.0
🎯 F1 Score: 0.7540983606557377
🌀 Confusion Matrix:
 [[ 0 15]
 [ 0 23]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [9]:
from sklearn.metrics import roc_auc_score
import torch.nn.functional as F

y_probs = []

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(device)
        outputs = model_swin(imgs)
        probs = F.softmax(outputs, dim=1)
        y_probs.extend(probs[:, 1].cpu().numpy())

roc_auc = roc_auc_score(y_true, y_probs)

print("🏆 ROC-AUC Score:", roc_auc)


🏆 ROC-AUC Score: 0.7971014492753623
